In [ ]:
CREATE OR REFRESH STREAMING TABLE ${bronze_table_name}
CLUSTER BY AUTO
AS SELECT *, _metadata AS source_metadata 
FROM STREAM read_files('${source_file_path}')

In [ ]:
CREATE OR REFRESH STREAMING TABLE ${silver_table_name} TBLPROPERTIES('delta.feature.variantType-preview' = 'supported')
CLUSTER BY AUTO
AS SELECT split_part(source_metadata.file_path, "/", ${ingestion_date_depth}) as ingestion_date, split_part(source_metadata.file_path, "/", ${billing_period_depth}) as billing_period, *, try_parse_json(tags) AS parsedTags 
FROM stream(${bronze_table_name})

In [ ]:
CREATE OR REPLACE MATERIALIZED VIEW ${gold_table_name} TBLPROPERTIES('delta.feature.variantType-preview' = 'supported')
CLUSTER BY AUTO
AS SELECT date, meterName, meterCategory, meterSubCategory, meterRegion, subscriptionName, resourceGroupName, parsedTags, quantity * unitPrice AS price FROM (
    SELECT *, RANK() OVER (PARTITION BY billing_period ORDER BY ingestion_date DESC) AS rank
    FROM actuals_silver
  ) WHERE rank = 1